## initiate

In [0]:
%run "./common_module"

In [0]:
fillna_value = dbutils.widgets.get("fillna_value")
print(fillna_value)

## read table

In [0]:
base_table = dbutils.widgets.get("base_table")
print(base_table)

In [0]:
winback = spark.table(base_table)
display(winback)

## drop useless rows

In [0]:
winback = winback.select('*').where(col("PO_PP_IND") == 'PO')
winback = winback.select('*').where(col("SUBSCR_TYPE") != 'D')


## drop useless column

In [0]:
winback = winback.drop("CURR_IMEI_BAND_FREQ","IMEI_TAC_PRICE","HS_RELEASE_TENURE","HS_MKT_RELEASE_TENURE","CURR_IMEI_TYPE" # imei-related are all overdated DATA
                       ,"P6M_MIN_NET_INV_AMT" # DUP MEANING
                       ,"LAST_CHANNEL_TYPE" # WRONG DATA
                       , "PO_PP_IND") # NO-NEED DATA


## transformation - correct data type

In [0]:
winback = winback.withColumn("Y_GA_FLAG",col("Y_GA_FLAG").cast("string"))
winback = winback.withColumn("AGE_BAND",col("AGE_BAND").cast("string"))
winback = winback.withColumn("AGE_BL35_IND",col("AGE_BL35_IND").cast("string"))

## categorical data: transformation - fix weird data

In [0]:
winback = winback.withColumn("CITY_NAME",when(col('CITY_NAME') == '臺中市    ', 'Taichung')\
.when(col('CITY_NAME') == '臺北市    ', 'Taipei')\
.when(col('CITY_NAME') == 'null', 'null')\
.when(col('CITY_NAME') == '臺東縣    ', 'Taitung')\
.when(col('CITY_NAME') == '金門縣    ', 'Kinmen')\
.when(col('CITY_NAME') == '高雄市    ', 'Kaoshiung')\
.when(col('CITY_NAME') == '澎湖縣    ', 'Penghu')\
.when(col('CITY_NAME') == '南投縣    ', 'Nantou')\
.when(col('CITY_NAME') == '新竹市    ', 'HsinchuCity')\
.when(col('CITY_NAME') == '屏東縣    ', 'Pingtong')\
.when(col('CITY_NAME') == '新竹縣    ', 'HsinchuCounty')\
.when(col('CITY_NAME') == '嘉義市    ', 'ChiayiCity')\
.when(col('CITY_NAME') == '基隆市    ', 'Keelung')\
.when(col('CITY_NAME') == '嘉義縣    ', 'ChiayiCounty')\
.when(col('CITY_NAME') == '雲林縣    ', 'Yunling')\
.when(col('CITY_NAME') == '宜蘭縣    ', 'Yilan')\
.when(col('CITY_NAME') == '臺南市    ', 'Tainan')\
.when(col('CITY_NAME') == '花蓮縣    ', 'Hualien')\
.when(col('CITY_NAME') == '苗栗縣    ', 'Miaoli')\
.when(col('CITY_NAME') == '連江縣    ', 'Lienchaing')\
.when(col('CITY_NAME') == '新北市    ', 'NewTaipei')\
.when(col('CITY_NAME') == '桃園市    ', 'Taoyuan')\
.when(col('CITY_NAME') == '彰化縣    ', 'Changhua')\

)

In [0]:
winback = winback.withColumn("SMS_PROM_IND",when(col('SMS_PROM_IND') == 'null', 'null')\
.when(col('SMS_PROM_IND') == 'Y', 'Y')\
.when(col('SMS_PROM_IND') == 'R', 'N')\
.when(col('SMS_PROM_IND') == 'N', 'N')\
.when(col('SMS_PROM_IND') == '1', 'Y')\
)

## numeric data: transformation - see statistical summary in numeric data

In [0]:
fillna_value = dbutils.widgets.get("fillna_value")
print(fillna_value)

In [0]:
winback = fillna_for_numeric_data(winback,['ID'],int(fillna_value))
display(winback)

## get_dummy

In [0]:
winback = winback.drop('DATA_MONTH')

In [0]:
winback = get_dummy(winback,['ID','Y_GA_FLAG'])
display(winback)

In [0]:
# %sql
# drop table cvm_s.ann_y22_iphone_ga_training_data_dummy

## drop useless dummy column

In [0]:
winback = winback.drop('SUBSCR_TYPE_v_', 'SMS_PROM_IND_n')

## write to DB

In [0]:
#write to database
winback.write.format('delta').saveAsTable('DB.table_dummy',mode = 'overwrite')